In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/bmaltais/kohya_ss.git
!wget https://raw.githubusercontent.com/yushan777/kohya_ss_vastai/main/!sdxl_kohya_vastai_no_config.ipynb
!mv '!sdxl_kohya_vastai_no_config.ipynb' 'kohya_ss'

Cloning into 'kohya_ss'...
remote: Enumerating objects: 9894, done.
remote: Counting objects: 100% (2175/2175), done.
remote: Compressing objects: 100% (172/172), done.
remote: Total 9894 (delta 2083), reused 2031 (delta 2001), pack-reused 7719
Receiving objects: 100% (9894/9894), 11.22 MiB | 19.60 MiB/s, done.
Resolving deltas: 100% (6869/6869), done.
--2023-12-19 07:47:22--  https://raw.githubusercontent.com/yushan777/kohya_ss_vastai/main/!sdxl_kohya_vastai_no_config.ipynb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16369 (16K) [text/plain]
Saving to: ‘!sdxl_kohya_vastai_no_config.ipynb’

!sdxl_kohya_vastai_ 100%[===================>]  15.99K  --.-KB/s    in 0.001s  

2023-12-19 07:47:22 (21.2 MB/s) - ‘!sdxl_kohya_vastai_no_config.ipynb’ saved [16369/1

In [ ]:
# 1. INSTALL DEPENDENCIES

!wget https://raw.githubusercontent.com/yushan777/kohya_ss_vastai/main/colors.py -q
from colors import bcolors
from IPython.display import clear_output

print(f"{bcolors.BOLD}{bcolors.GREEN}Installing prerequisites...{bcolors.ENDC}")
!apt-get update -y && apt-get install -y -qq libgl1
!sudo apt update -y && sudo apt install -y -qq python3-tk
clear_output(wait=True)

print(f"{bcolors.BOLD}{bcolors.GREEN}Installing main dependencies - this will take a few minutes...{bcolors.ENDC}")

%pip install gdown
!chmod +x setup.sh
!./setup.sh

print(f"{bcolors.BOLD}{bcolors.GREEN}Finished installing dependencies.{bcolors.ENDC}")


Installing main dependencies - this will take a few minutes...
chmod: cannot access 'setup.sh': No such file or directory
/bin/bash: line 1: ./setup.sh: No such file or directory
Finished installing dependencies.


In [ ]:
# 2. Configure Accelerate With defaults and bf16 precision
!accelerate config default --mixed_precision "bf16"

/bin/bash: line 1: accelerate: command not found


In [ ]:
# 3. VARIABLES
# The following variables can be changed according to your project preferences
token_word = "ohwx"
class_word = "panda"
training_repeats = 3
training_root_dir = "training_images"
regularization_root_dir = "reg_images"
models_dir = "training_models"
project_name = "SDXL1"
output_dir = "trained_models"

In [ ]:
# 4. CREATE DATASET FOLDERS & MODEL FOLDER

# ================================================================
# your training and reg image subfolders will be named according to what is set in previous cell:
# so if you stuck with the defaults, they would be for example :
# "training_images/40_ohwx person"
# "reg_images/1_person"
training_dir = f'{training_root_dir}/{training_repeats}_{token_word} {class_word}'
reg_dir = f'{regularization_root_dir}/1_{class_word}'

# back to parent folder
%cd /workspace/kohya_ss

import os
if os.path.exists(training_dir) == False:
  os.makedirs(training_dir)
  print(f'{training_dir} Created.')
else:
  print(f'{training_dir} already exists.')

if os.path.exists(reg_dir) == False:
  os.makedirs(reg_dir)
  print(f'{reg_dir} Created.')
else:
  print(f'{reg_dir} already exists.')

if os.path.exists(models_dir) == False:
  os.makedirs(models_dir)
  print(f'{models_dir} Created.')
else:
  print(f'{models_dir} already exists.')

# Create prompt file use for samples during training.
# first 2 prompts are the same but one has CFG=1 and the other CFG=7
# if subject likeness is strong and regular even at CFG=1 then it is an good indicator that it is overfitted
# third prompt is to check how well it responds to styling.
lines = [
    f"a photo of {token_word} {class_word} --w 1024 --h 1024 --l 7, --s 20 --d 1234567890\n",
    f"a photo of {token_word} {class_word} --w 1024 --h 1024 --l 1, --s 20 --d 1234567890\n",
    f"a portrait of {token_word} {class_word} in the style of Rembrandt --w 1024 --h 1024 --l 6, --s 20 --d 1234567890\n"
]

# Create and write to the text file
with open("prompt.txt", "w") as file:
    file.writelines(lines)

print("File 'prompt.txt' has been created.")

[Errno 2] No such file or directory: '/workspace/kohya_ss'
/content
training_images/3_ohwx panda Created.
reg_images/1_panda Created.
training_models already exists.
File 'prompt.txt' has been created.


In [ ]:
from google.colab import drive
import os
import shutil
import zipfile

# Mount your Google Drive
drive.mount('/content/drive')

# Your Google Drive path to the folder containing the training images
drive_folder_path = '/content/drive/MyDrive/DL_Project/kungfu15shortlist'

# Path to your training directory in Colab
training_dir = '/content/training_images'

# Create the training directory if it doesn't exist
os.makedirs(training_dir, exist_ok=True)

# Copy all files from the Drive folder to the local training directory
for filename in os.listdir(drive_folder_path):
    file_path = os.path.join(drive_folder_path, filename)
    if os.path.isfile(file_path):
        shutil.copy(file_path, training_dir)

# Unzip images in the training directory
for file in os.listdir(training_dir):
    if file.endswith('.zip'):
        with zipfile.ZipFile(os.path.join(training_dir, file), 'r') as zip_ref:
            zip_ref.extractall(training_dir)
        os.remove(os.path.join(training_dir, file))

# Now your training images should be in the training directory


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import drive
import os
import shutil
import zipfile

# Mount your Google Drive
drive.mount('/content/drive')

# Your Google Drive path to the folder containing the regular images
drive_folder_path = '/content/drive/MyDrive/DL_Project/kungfu_classNew'

# Path to your regular images directory in Colab
reg_dir = '/content/reg_images'

# Create the regular images directory if it doesn't exist
os.makedirs(reg_dir, exist_ok=True)

# Copy all files from the Drive folder to the local regular images directory
for filename in os.listdir(drive_folder_path):
    file_path = os.path.join(drive_folder_path, filename)
    if os.path.isfile(file_path):
        shutil.copy(file_path, reg_dir)

# Unzip images in the regular images directory
for file in os.listdir(reg_dir):
    if file.endswith('.zip'):
        with zipfile.ZipFile(os.path.join(reg_dir, file), 'r') as zip_ref:
            zip_ref.extractall(reg_dir)
        os.remove(os.path.join(reg_dir, file))

# Now your regular images should be in the regular images directory


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 7. Check Dataset Folders
# ====================================================
# remove any non-image files & warn if any additional folders exist
import os
import shutil
from glob import glob
folder_path = f'{training_dir}'

# Get a list of all files in the folder
files = glob(folder_path + '/*', recursive=False)

# Iterate over the files and delete the ones that are not JPG or PNG
for file_path in files:
    if not (file_path.endswith('.jpg') or file_path.endswith('.png')):
        if os.path.isfile(file_path):
            os.remove(file_path)
        elif os.path.isdir(file_path):
            print(f'{bcolors.BOLD}{bcolors.RED} Unexpected folder: \'{file_path}\' was found in training images folder path.  Check and remove it.{bcolors.ENDC}')

# force remove hidden .ipynb_checkpoints folder in images folder.
if os.path.exists(f'{folder_path}/.ipynb_checkpoints'):
    shutil.rmtree(f'{folder_path}/.ipynb_checkpoints')

# ====================================================
# delete any non-image files & warn if any additional folders

folder_path = f'{reg_dir}'

# Get a list of all files in the folder
files = glob(folder_path + '/*', recursive=False)

# Iterate over the files and delete the ones that are not JPG or PNG
for file_path in files:
    if not (file_path.endswith('.jpg') or file_path.endswith('.png')):
        if os.path.isfile(file_path):
            os.remove(file_path)
        elif os.path.isdir(file_path):
            print(f'{bcolors.BOLD}{bcolors.RED} Unexpected folder: \'{file_path}\' was found in training images folder path.  Check and remove it.{bcolors.ENDC}')

# force remove hidden .ipynb_checkpoints folder in images folder.
if os.path.exists(f'{folder_path}/.ipynb_checkpoints'):
    shutil.rmtree(f'{folder_path}/.ipynb_checkpoints')

 Unexpected folder: '/content/training_images/3_ohwx panda' was found in training images folder path.  Check and remove it.
 Unexpected folder: '/content/reg_images/1_panda' was found in training images folder path.  Check and remove it.


In [ ]:
# 8. Download SDXL 1.0 (0.9VAE) base model (the one with 0.9vae baked in instead of 1.0vae
from huggingface_hub import hf_hub_download

# Specify the repository ID, the filename, and the desired download directory
repo_id = 'stabilityai/stable-diffusion-xl-base-1.0'
filename = 'sd_xl_base_1.0_0.9vae.safetensors'
download_directory = 'training_models'

# Download the file to the training_models dir, as a regular file (not symlink)
base_model = hf_hub_download(repo_id=repo_id, filename=filename, local_dir=download_directory, local_dir_use_symlinks=False)
print(base_model)


sd_xl_base_1.0_0.9vae.safetensors:   0%|          | 0.00/6.94G [00:00<?, ?B/s]

training_models/sd_xl_base_1.0_0.9vae.safetensors


In [ ]:
!pwd

/content


In [ ]:
%cd kohya_ss/

/content/kohya_ss


In [ ]:
!pwd

/content/kohya_ss


In [ ]:
!pip install git+https://github.com/huggingface/diffusers
!pip install -qq accelerate transformers ftfy

  Cloning https://github.com/huggingface/diffusers to /tmp/pip-req-build-0xycegz4
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-0xycegz4
  Resolved https://github.com/huggingface/diffusers to commit bf40d7d82a732a35e0f5b907e59064ee080cde9f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.25.0.dev0-py3-none-any.whl size=1833134 sha256=8ab79ab060b72f32e192181f0c6c60726686289dabc1b7de0534080f770cffc0
  Stored in directory: /tmp/pip-ephem-wheel-cache-sav_2ttt/wheels/f7/7d/99/d361489e5762e3464b3811bc629e94cf5bf5ef44dd5c3c4d52
Successfully built diffusers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 8.3 MB/s eta 0:00:00


In [ ]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.2 MB/s eta 0:00:00


In [ ]:
!pip install voluptuous


In [ ]:
# 9. START TRAINING

max_steps = 2000
learning_rate = 1e-5
learning_rate_te1 = 1e-7
learning_rate_te2 = 1e-7
save_every_n_steps = 1000

print(base_model)
!accelerate launch --num_cpu_threads_per_process=2 "./sdxl_train.py" \
  --pretrained_model_name_or_path="/content/kohya_ss/training_models/sd_xl_base_1.0_0.9vae.safetensors" \
  --train_data_dir="/content/kohya_ss/training_images" \
  --reg_data_dir="/content/kohya_ss/reg_images" \
  --output_dir="/content/kohya_ss/trained_models" \
  --output_name="{project_name}" \
  --save_model_as="safetensors" \
  --train_batch_size=2 \
  --max_train_steps={max_steps} \
  --save_every_n_steps={save_every_n_steps} \
  --optimizer_type="adafactor" \
  --optimizer_args scale_parameter=False relative_step=False warmup_init=False \
  --xformers \
  --cache_latents \
  --lr_scheduler="constant_with_warmup" \
  --lr_warmup_steps=100 \
  --learning_rate="{learning_rate}" \
  --learning_rate_te1="{learning_rate_te1}" \
  --learning_rate_te2="{learning_rate_te2}" \
  --max_grad_norm=0.0 \
  --train_text_encoder \
  --resolution="512,512" \
  --save_precision="bf16" \
  --save_n_epoch_ratio=1 \
  --max_data_loader_n_workers=1 \
  --persistent_data_loader_workers \
  --mixed_precision="bf16" \
  --full_bf16 \
  --logging_dir="logs" \
  --log_prefix="last" \
  --gradient_checkpointing \
  --min_snr_gamma=0 \
  --noise_offset=0.0357 \
  --sample_sampler="euler_a" \
  --sample_prompts="prompt.txt" \
  --sample_every_n_steps=100

# Training finished...
# List all checkpoints
print("Trained models:\n")
!ls trained_models -1

training_models/sd_xl_base_1.0_0.9vae.safetensors
The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2023-12-19 08:35:00.568224: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-19 08:35:00.568285: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-19 08:35:00.569607: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to regi